In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from aif360.datasets import BinaryLabelDataset,CompasDataset

In [4]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from functions import*

In [5]:
import os
path=os.path.dirname(os.getcwd())

In [6]:
class ROCpostprocess:
    def __init__(self,X_val,y_val,var_list,prediction_model,favorable_label):
        self.X_val =X_val
        self.y_val =y_val
        self.model = prediction_model
        self.positive_index = 1 # positive label
        self.var_list = var_list
        self.var_dim=len(self.var_list)
        self.ROC = self.buildROCusingval()
        self.favorable_label = favorable_label

    def buildbinarydata(self,X,y):
        df=pd.DataFrame(np.concatenate((X,y.reshape(-1,1)), axis=1),columns=self.var_list+['S','W','Y'])
        binaryLabelDataset = BinaryLabelDataset(
                            # favorable_label=self.favorable_label,
                            # unfavorable_label=0,
                            df=df[self.var_list+['S','W','Y']], #df_test.drop('X',axis=1), #[x_list+['S','W','Y']],
                            label_names=['Y'],
                            instance_weights_name=['W'],
                            protected_attribute_names=['S'],
                            privileged_protected_attributes=[np.array([1.0])],
                            unprivileged_protected_attributes=[np.array([0.])])
        return binaryLabelDataset,df

    def buildROCusingval(self):
        dataset_val = self.buildbinarydata(self.X_val,self.y_val)[0]
        dataset_val_pred = dataset_val.copy(deepcopy=True)
        dataset_val_pred.scores = self.model.predict_proba(dataset_val.features[:,0:self.var_dim])[:,self.positive_index].reshape(-1,1)
        privileged_groups = [{'S': 1}]
        unprivileged_groups = [{'S': 0}]
        # Metric used (should be one of allowed_metrics)
        metric_name = "Statistical parity difference"
        # Upper and lower bound on the fairness metric used
        metric_ub = 0.05
        metric_lb = -0.05
        ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                        privileged_groups=privileged_groups, 
                                        low_class_thresh=0.01, high_class_thresh=0.99,
                                        num_class_thresh=50, num_ROC_margin=10,
                                        metric_name=metric_name,
                                        metric_ub=metric_ub, metric_lb=metric_lb)
        ROC = ROC.fit(dataset_val, dataset_val_pred)
        print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
        print("Optimal ROC margin = %.4f" % ROC.ROC_margin)
        return ROC

    def postprocess(self,X_test,y_test,tv_origin): # the tv distance won't change
        dataset_test_pred,df_test = self.buildbinarydata(X_test,y_test) #.copy(deepcopy=True)
        dataset_test_pred.scores = self.model.predict_proba(X_test[:,0:self.var_dim])[:,self.positive_index].reshape(-1,1)
        dataset_test_pred_transf = self.ROC.predict(dataset_test_pred)
        y_pred = dataset_test_pred_transf.labels
        # return dataset_test_pred_transf.convert_to_dataframe()[0]

        di = DisparateImpact_postprocess(df_test,y_pred,favorable_label=self.favorable_label)
        f1_macro = f1_score(df_test['Y'], y_pred, average='macro',sample_weight=df_test['W'])
        f1_micro = f1_score(df_test['Y'], y_pred, average='micro',sample_weight=df_test['W'])
        f1_weighted = f1_score(df_test['Y'], y_pred, average='weighted',sample_weight=df_test['W'])
        new_row=pd.Series({'DI':di,'f1 macro':f1_macro,'f1 micro':f1_micro,'f1 weighted':f1_weighted,
                           'TV distance':tv_origin,'method':'ROC'})
        return new_row.to_frame().T

In [7]:
class Projpostprocess:
    
    def __init__(self,X_test,y_test,x_list,var_list,prediction_model,K,e,thresh,favorable_label=1):
        self.model = prediction_model
        self.K=K
        self.e=e
        self.x_list=x_list
        self.var_list=var_list
        self.var_dim=len(var_list)
        self.favorable_label = favorable_label

        df_test=pd.DataFrame(np.concatenate((X_test,y_test.reshape(-1,1)), axis=1),columns=var_list+['S','W','Y'])
        df_test=df_test.groupby(by=var_list+['S','Y'],as_index=False).sum()
        if len(x_list)>1:
            df_test['X'] = list(zip(*[df_test[c] for c in x_list]))
            self.x_range=sorted(set(df_test['X']))
            weight=list(1/(df_test[x_list].max()-df_test[x_list].min())) # because 'education-num' range from 1 to 16 while others 1 to 4
            self.C=c_generate_higher(self.x_range,weight)
        else:
            df_test['X']=df_test[x_list]
            self.x_range=sorted(set(df_test['X']))
            self.C=c_generate(self.x_range)
        self.df_test = df_test
        self.var_range=list(pd.pivot_table(df_test,index=var_list,values=['S','W','Y']).index)
        self.distribution_generator()
        
        if thresh == 'auto':
            self.thresh_generator()
        else:
            self.thresh=thresh

    def distribution_generator(self):
        bin=len(self.x_range)
        dist=rdata_analysis(self.df_test,self.x_range,'X')
        
        dist['v']=[(dist['x_0'][i]-dist['x_1'][i])/dist['x'][i] for i in range(bin)]
        
        dist['t_x']=dist['x'] # #dist['x'] #dist['x_0']*0.5+dist['x_1']*0.5 
        self.px=np.matrix(dist['x']).T
        self.ptx=np.matrix(dist['t_x']).T
        if np.any(dist['x_0']==0): 
            self.p0=np.matrix((dist['x_0']+1.0e-9)/sum(dist['x_0']+1.0e-9)).T
        else:
            self.p0=np.matrix(dist['x_0']).T 
        if np.any(dist['x_1']==0):
            self.p1=np.matrix((dist['x_1']+1.0e-9)/sum(dist['x_1']+1.0e-9)).T
        else:
            self.p1=np.matrix(dist['x_1']).T 
        self.V=np.matrix(dist['v']).T
        self.tv_origin=sum(abs(dist['x_0']-dist['x_1']))/2
        # return px,ptx,V,p0,p1
    
    def coupling_generator(self,method,para=None):
        if method == 'unconstrained':
            coupling=baseline(self.C,self.e,self.px,self.ptx,self.K)
        elif method == 'barycentre':
            coupling=baseline(self.C,self.e,self.p0,self.p1,self.K)
        elif method == 'partial':
            coupling=partial_repair(self.C,self.e,self.px,self.ptx,self.V,para,self.K)
        return coupling

    def postprocess(self,method,para=None):
        if method == 'origin':
            y_pred=self.model.predict(np.array(self.df_test[self.var_list]))
            tv = self.tv_origin
        else:
            coupling = self.coupling_generator(method,para)
            if (method == 'unconstrained') or (method == 'partial'):
                y_pred=postprocess(self.df_test,coupling,self.x_list,self.x_range,self.var_list,self.var_range,self.model,self.thresh)
                tv=assess_tv(self.df_test,coupling,self.x_range,self.x_list,self.var_list)
                if (para != None) and (method == 'partial'):
                    method = method+'_'+str(para)
            elif method == 'barycentre':
                y_pred,tv=postprocess_bary(self.df_test,coupling,self.x_list,self.x_range,self.var_list,self.var_range,self.model,self.thresh)
            else:
                print('Unknown method')

        di = DisparateImpact_postprocess(self.df_test,y_pred,favorable_label=self.favorable_label)
        f1_macro = f1_score(self.df_test['Y'], y_pred, average='macro',sample_weight=self.df_test['W'])
        f1_micro = f1_score(self.df_test['Y'], y_pred, average='micro',sample_weight=self.df_test['W'])
        f1_weighted = f1_score(self.df_test['Y'], y_pred, average='weighted',sample_weight=self.df_test['W'])

        new_row=pd.Series({'DI':di,'f1 macro':f1_macro,'f1 micro':f1_micro,'f1 weighted':f1_weighted,
                           'TV distance':tv,'method':method})
        return new_row.to_frame().T
    
    def thresh_generator(self):
        num_thresh = 10
        ba_arr = np.zeros(num_thresh)
        ba_arr1 = np.zeros(num_thresh)
        class_thresh_arr = np.linspace(0.01, 0.1, num_thresh)
        coupling=self.coupling_generator('partial',para=1e-2)
    
        for idx, thresh in enumerate(class_thresh_arr):
            y_pred=postprocess(self.df_test,coupling,self.x_list,self.x_range,self.var_list,self.var_range,self.model,thresh)
            ba_arr[idx] = DisparateImpact_postprocess(self.df_test,y_pred,favorable_label=self.favorable_label)
            ba_arr1[idx] = f1_score(self.df_test['Y'], y_pred, average='macro',sample_weight=self.df_test['W'])

        best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
        best_thresh = class_thresh_arr[best_ind]
        print("Optional threshold = ",class_thresh_arr)
        print("Disparate Impact = ",ba_arr)
        print("f1 scores = ",ba_arr1)
        self.thresh = best_thresh

In [8]:
pa = 'race'
label_map = {1.0: 'Did recid.', 0.0: 'No recid.'}
protected_attribute_maps = {1.0: 'Caucasian', 0.0: 'African-American'}
favorable_label = 0
privileged_groups = [{pa: 1}]
unprivileged_groups = [{pa: 0}]
cd = CompasDataset(protected_attribute_names=[pa],privileged_classes=[['Caucasian'],[1]], 
                    metadata={'label_map': label_map,'protected_attribute_maps': protected_attribute_maps},
                    features_to_drop=['age', 'sex', 'c_charge_desc'])
train,test = cd.split([0.6], shuffle=True) #len(test.instance_names) = 2057
var_list = cd.feature_names.copy()
var_list.remove(pa)
var_dim=len(var_list)

K=200
e=0.01
thresh=0.05

messydata=cd.convert_to_dataframe()[0]
messydata=messydata.rename(columns={pa:'S',cd.label_names[0]:'Y'})
messydata=messydata[(messydata['S']==1)|(messydata['S']==0)]
for col in var_list+['S','Y']:
    messydata[col]=messydata[col].astype('category')
messydata['W']=cd.instance_weights
X=messydata[var_list+['S','W']].to_numpy() # [X,S,W]
y=messydata['Y'].to_numpy() #[Y]
tv_dist=dict()
for x_name in var_list:
    x_range_single=list(pd.pivot_table(messydata,index=x_name,values=['W'])[('W')].index) 
    dist=rdata_analysis(messydata,x_range_single,x_name)
    tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
x_list=[]
for key,val in tv_dist.items():
    if val>0.1:
        x_list+=[key]        
tv_dist

{'juv_fel_count': 0.03210337325453563,
 'juv_misd_count': 0.04323143324022939,
 'juv_other_count': 0.021763780679615215,
 'priors_count': 0.12622233191661625,
 'age_cat=25 - 45': 0.054431947619680315,
 'age_cat=Greater than 45': 0.13519019921101838,
 'age_cat=Less than 25': 0.08075825159133806,
 'c_charge_degree=F': 0.07840757396162046,
 'c_charge_degree=M': 0.07840757396162046}

In [9]:
x_list

['priors_count', 'age_cat=Greater than 45']

In [10]:
methods=['origin','unconstrained','barycentre','partial','ROC'] # Place ROC in the end
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(10):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    projpost = Projpostprocess(X_test,y_test,x_list,var_list,clf,K,e,thresh,favorable_label)
    for method in methods[:-1]:
        # report = pd.concat([report,projpost.postprocess(method,para=1e-2)], ignore_index=True)
        report = pd.concat([report,projpost.postprocess(method,para=1e-3)], ignore_index=True)

    ROCpost = ROCpostprocess(X_val,y_val,var_list,clf,favorable_label) # use validation set to train a ROC model
    report = pd.concat([report,ROCpost.postprocess(X_test,y_test,tv_origin=projpost.tv_origin)], ignore_index=True)

report.to_csv(path+'/data/report_postprocess_compas_'+str(pa)+'.csv',index=None)

Optimal classification threshold (with fairness constraints) = 0.6700
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.6900
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.6700
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.7100
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.1900
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.1900
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.7300
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.7100
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.7500
Optimal ROC margin = 0.0000
Optimal classification threshold (with fairness constraints) = 0.7300
Optimal ROC margin = 0.0000


In [35]:
# report.to_csv(path+'/data/report_postprocess_compas_'+str(pa)+'_'+str(thresh)+'.csv',index=None)

In [23]:
report

,DI,f1 macro,f1 micro,f1 weighted,TV distance,method
0,0.813009,0.655885,0.667882,0.662157,0.174339,origin
1,0.791119,0.659004,0.665857,0.663723,0.166751,unconstrained
2,0.671128,0.60937,0.610774,0.607084,0.000002,barycentre
3,0.973863,0.579982,0.579992,0.579783,0.023015,partial_0.001
4,0.920506,0.471651,0.593358,0.496403,0.174339,ROC
5,0.769508,0.653563,0.656946,0.657487,0.183173,origin
6,0.765398,0.656064,0.658971,0.659688,0.178038,unconstrained
7,0.763133,0.57506,0.579992,0.569812,0.000051,barycentre
8,1.03774,0.568569,0.569056,0.566906,0.021052,partial_0.001
9,0.910141,0.487691,0.595383,0.514614,0.183173,ROC


In [20]:
valpost = Projpostprocess(X_val,y_val,x_list,var_list,clf,K,e,'auto')
valpost.thresh

Optional threshold =  [0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1 ]
Disparate Impact =  [1.01773973 1.07230386 1.07230386 1.09176684 1.09176684 1.09176684
 1.09176684 1.09176684 1.09176684 1.09176684]
f1 scores =  [0.57720086 0.59726485 0.59726485 0.6057577  0.6057577  0.6057577
 0.6057577  0.6057577  0.6057577  0.6057577 ]


0.04000000000000001

In [10]:
# Compute average feature importance
importance=[]
for ignore in range(10):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    importance.append(list(clf.feature_importances_))
importance=np.array(importance)
print("features", var_list)
print("mean importances", importance.mean(axis=0))

features ['juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'age_cat=25 - 45', 'age_cat=Greater than 45', 'age_cat=Less than 25', 'c_charge_degree=F', 'c_charge_degree=M']
mean importances [0.02887228 0.05502374 0.08083862 0.58488193 0.02901493 0.08051421
 0.07832186 0.03111952 0.0314129 ]
